In [2]:
import os
from dotenv import find_dotenv, load_dotenv
load_dotenv(find_dotenv("req.env"), override=True)

#os.environ.get("GOOGLE_API_KEY")

True

In [3]:
import google.generativeai as genai
for model in genai.list_models():
    print(model.name)

models/chat-bison-001
models/text-bison-001
models/embedding-gecko-001
models/gemini-1.0-pro-latest
models/gemini-1.0-pro
models/gemini-pro
models/gemini-1.0-pro-001
models/gemini-1.0-pro-vision-latest
models/gemini-pro-vision
models/gemini-1.5-pro-latest
models/gemini-1.5-pro-001
models/gemini-1.5-pro
models/gemini-1.5-flash-latest
models/gemini-1.5-flash-001
models/gemini-1.5-flash
models/embedding-001
models/text-embedding-004
models/aqa


In [5]:
from langchain_google_genai import ChatGoogleGenerativeAI
llm =ChatGoogleGenerativeAI(model='gemini-pro', temperature=0.9)
response=llm.invoke("What is the most popular song in Black Album")
print(response.content)

Enter Sandman


In [6]:
def load_document(file):
    import os
    name, extension = os.path.splitext(file)
    
    if extension == '.pdf':
        from langchain.document_loaders import PyPDFLoader
        print(f'Loading {file}')
        try:
            loader = PyPDFLoader(file_path=file)
            data = loader.load()
            return data
        except Exception as e:
            print(f"Error loading PDF file: {e}")
            return None
    elif extension == '.docx':
        from langchain.document_loaders import Docx2txtLoader
        print(f'Loading {file}')
        try:
            loader = Docx2txtLoader(file)
            data = loader.load()
            return data
        except Exception as e:
            print(f"Error loading DOCX file: {e}")
            return None
    else:
        print('Document format is not supported')
        return None


In [7]:
def chunk_data(data, chunk_size=256):
    from langchain.text_splitter import RecursiveCharacterTextSplitter
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=0)
    chunks=text_splitter.split_documents(data)
    return chunks

# Embedding and Uploading to a Vector Database (Pinecone)

In [20]:
def insert_or_fetch_embeddings1(index_name, chunks):
    import pinecone
    from langchain_community.vectorstores import Pinecone
    from langchain_google_genai import GoogleGenerativeAIEmbeddings 
    from pinecone import PodSpec

    pc= pinecone.Pinecone()
    embeddings = GoogleGenerativeAIEmbeddings(model='models/embedding-001')  

    pc = pinecone.Pinecone()
    
    if index_name in pc.list_indexes():
        print(f'Index {index_name} already exists. Loading embeddings...', end='')
        vector_store = Pinecone.from_existing_index(index_name, embeddings)
        print('ok')
    else:
        print(f'Creating index {index_name} and embeddings...', end='')
        pc.create_index(
            name=index_name,
            dimension=768,
            metric='cosine',
            spec=PodSpec(environment = 'gcp-starter')
        )
        vector_store = Pinecone.from_documents(chunks, embeddings, index_name=index_name)
        print('Ok')

    return vector_store

In [12]:
def delete_pinecone_index(index_name='all'):
    import pinecone
    pc =pinecone.Pinecone()
    if index_name == 'all':
        indexes = pc.list_indexes().names()
        print('Deleting all indexes...')
        for index in indexes:
            pc.delete_index(index)
        print('Ok')
    else:
        print(f'Deleting index {index_name} ...', end='')
        

In [13]:
def ask_and_get_answer(vector_store, q):
    from langchain.chains import RetrievalQA
    from langchain_google_genai import ChatGoogleGenerativeAI
    
    if vector_store is None:
        raise ValueError("Vector store is None, cannot proceed with the retrieval.")

    llm = ChatOpenAI(model='gemini-pro', temperature=1)
    
    retriever = vector_store.as_retriever(search_type='similarity', search_kwargs={'k': 3})
    # k equals 3 means that it will return the three most similar chunks to the user's query.
    
    chain = RetrievalQA.from_chain_type(llm=llm, chain_type='stuff', retriever=retriever)

    answer = chain.run(q)
    return answer

In [14]:
data = load_document('den.pdf')

print(f'we have {len(data)} pages')

Loading den.pdf
we have 1 pages


In [15]:
chunks = chunk_data(data)
print(len(chunks))
print(chunks[10].page_content)

14
testing. She used unit test, selenium test, white box test, and black box test. The 
company name is BİMSER. She has new internship about Python backend development.


In [16]:
delete_pinecone_index()

Deleting all indexes...
Ok


In [21]:
index_name= 'askdocument'
vector_store =insert_or_fetch_embeddings1(index_name, chunks)

Creating index askdocument and embeddings...

ForbiddenException: (403)
Reason: Forbidden
HTTP response headers: HTTPHeaderDict({'content-type': 'text/plain; charset=utf-8', 'x-pinecone-api-version': '2024-07', 'access-control-allow-origin': '*', 'vary': 'origin,access-control-request-method,access-control-request-headers', 'access-control-expose-headers': '*', 'X-Cloud-Trace-Context': '793188ca8d30f244a35b63988efa17c9', 'Date': 'Mon, 29 Jul 2024 11:04:46 GMT', 'Server': 'Google Frontend', 'Content-Length': '142', 'Via': '1.1 google', 'Alt-Svc': 'h3=":443"; ma=2592000,h3-29=":443"; ma=2592000'})
HTTP response body: {"error":{"code":"FORBIDDEN","message":"Request failed. Your free plan supports 1 starter indexes. Use a different index type."},"status":403}
